In [1]:
# Audio Waveform Viewer which support windows close process

import pyaudio
import struct
import numpy as np 
%matplotlib qt
import matplotlib.pyplot as plt
import time
CHUNK = 1024 * 4
CHANNELS = 1
FORMAT = pyaudio.paInt16
RATE = 44100 # Sample per second. 44 kHz is most common value



p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)
def audio_close_procedure():
    stream.stop_stream()
    stream.close()
    print('stream stopped and close') 
    p.terminate()        
    print('pyaudio terminate()')                 

    
def plt_handle_close(evt):
    print('Closed Figure!')
    audio_close_procedure()

    
    
fig, ax = plt.subplots() 
x = np.arange(0, 2 *CHUNK, 2)
line, = ax.plot(x, np.random.rand(CHUNK))
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])
fig.canvas.mpl_connect('close_event', plt_handle_close)
print('stream started')

while True:    
    try:
        data = stream.read(CHUNK, exception_on_overflow = False)
        data_int = np.array( struct.unpack(str(2*CHUNK) +'B',data), dtype='b')[::2] + 128 
        line.set_ydata(data_int)
        fig.canvas.draw()
        fig.canvas.flush_events()
    except: 
        audio_close_procedure()
        break



stream started
Closed Figure!
stream stopped and close
pyaudio terminate()
stream stopped and close
pyaudio terminate()


In [1]:

# Audio Spectrum Analyzer 

import pyaudio
import struct
import numpy as np 
%matplotlib qt
import matplotlib.pyplot as plt
import time

from scipy.fftpack import fft

CHUNK = 1024 * 2
CHANNELS = 1
FORMAT = pyaudio.paInt16
RATE = 44100 # Sample per second. 44 kHz is most common value



p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)
frame_count = 0
start_time = time.time()
def audio_close_procedure():
    frame_rate = frame_count / (time.time() - start_time)
    print('average frame rate = {:.0f} FPS'.format(frame_rate))
    
    stream.stop_stream()
    stream.close()
    print('stream stopped and close') 
    p.terminate()        
    print('pyaudio terminate()')                 

    
def plt_handle_close(evt):
    print('Closed Figure!')
    audio_close_procedure()

    
    
#fig, ax = plt.subplots() 
fig, (ax, ax2 ) = plt.subplots(2, figsize=(15,8))

# Variable for plotting
x = np.arange(0, 2 *CHUNK, 2)
x_fft = np.linspace(0, RATE, CHUNK)

#line, = ax.plot(x, np.random.rand(CHUNK))
line, = ax.plot(x, np.random.rand(CHUNK),'-', lw=2)
line.set_linewidth(0.5)
#line_fft, = ax2.plot(x_fft, np.random.rand(CHUNK), '-', lw=2)
line_fft, = ax2.semilogx(x_fft, np.random.rand(CHUNK), '-', lw=2)
line_fft.set_linewidth(1)

ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
#ax.set_xlim(0, CHUNK)
ax.set_xlim(0, 2*CHUNK)
ax2.set_xlim(20, RATE / 2)

plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])



fig.canvas.mpl_connect('close_event', plt_handle_close)
print('stream started')

while True:    
    try:
        data = stream.read(CHUNK, exception_on_overflow = False)
        
        #data_int = np.array( struct.unpack(str(2*CHUNK) +'B',data), dtype='b')[::2] + 128 
        #line.set_ydata(data_int)
        data_int = struct.unpack(str(2*CHUNK) +'B',data)
        data_np = np.array( data_int, dtype='b')[::2] + 128 
        line.set_ydata(data_np)
        
        
        y_fft = fft(data_int)
        line_fft.set_ydata(np.abs(y_fft[0:CHUNK])*2 / (256*CHUNK) )
        
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        frame_count = frame_count + 1 
        
    except: 
        audio_close_procedure()
        break





stream started
Closed Figure!
average frame rate = 8 FPS
stream stopped and close
pyaudio terminate()
average frame rate = 8 FPS
stream stopped and close
pyaudio terminate()


In [3]:
fft(np.sin(2*x))

array([0.31625928+0.j        , 0.31625882+0.00088698j,
       0.31625744+0.00177397j, ..., 0.31625514-0.00266097j,
       0.31625744-0.00177397j, 0.31625882-0.00088698j])

In [6]:
plt.plot(np.abs(fft(np.sin(2*x))))